In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/MyDrive/PHD/SemEval2023

/content/gdrive/MyDrive/PHD/SemEval2023


In [ ]:
import pandas as pd

data = pd.read_csv("Data/starting_ki/train_all_tasks.csv")
dfa = pd.read_csv("Data/dev_task_a_entries.csv")
# dfb = pd.read_csv("Data/dev_task_b_entries.csv")
dfc = pd.read_csv("Data/dev_task_c_entries.csv")

In [ ]:
data

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,"Damn, this writing was pretty chaotic",not sexist,none,none
1,sexism2022_english-2367,"Yeah, and apparently a bunch of misogynistic v...",not sexist,none,none
2,sexism2022_english-3073,How the FUCK is this woman still an MP!!!???,not sexist,none,none
3,sexism2022_english-14895,Understand. Know you're right. At same time I ...,not sexist,none,none
4,sexism2022_english-4118,Surprized they didn't stop and rape some women,not sexist,none,none
...,...,...,...,...,...
13995,sexism2022_english-14185,complexes like the 'nice chicks' that go after...,sexist,2. derogation,2.1 descriptive attacks
13996,sexism2022_english-18629,"""GRAPHIC Germany - Muslim ""refugee"" stabbing h...",not sexist,none,none
13997,sexism2022_english-3320,Lol I imagine there would be simps that are li...,not sexist,none,none
13998,sexism2022_english-13309,"It's not, the girls I go on dates with don't k...",not sexist,none,none


In [ ]:
data[["text", "label_sexist"]].groupby("label_sexist").count()

,text
label_sexist,
not sexist,10602
sexist,3398


In [ ]:
label_values = data["label_sexist"].unique()
label_values

array(['not sexist', 'sexist'], dtype=object)

In [ ]:
dfa

,rewire_id,text
0,sexism2022_english-11999,I do not try to hard. I do what guys who get w...
1,sexism2022_english-9895,That sucks. We elected a Navy Seal in Missouri...
2,sexism2022_english-11623,Walk away and never look back. If she reaches ...
3,sexism2022_english-19035,basically I placed more value on her than I sh...
4,sexism2022_english-895,"NYT and Asian ""part ways."" But why on earth wa..."
...,...,...
1995,sexism2022_english-13861,"None ,but some of the girls I bang are on ecst..."
1996,sexism2022_english-6864,"It's a constant isnt it, whenever women fall b..."
1997,sexism2022_english-17934,yep. No fapping means you have to chase the va...
1998,sexism2022_english-4138,I didn't say you had to stay with a crazy bitc...


In [ ]:
texts = dfa['text'].values
# labels = [1 if d=="sexist" else 0 for d in data['label_sexist'].values]

# Prepare Data

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 5.8 MB 32.6 MB/s 
     |████████████████████████████████| 182 kB 76.0 MB/s 
     |████████████████████████████████| 7.6 MB 64.1 MB/s 


In [ ]:
import transformers

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

print('Original Text: ', texts[0], '\n')
print('Tokenized Text: ', tokenizer.tokenize(texts[0]), '\n')
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(texts[0])))


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Original Text:  Damn, this writing was pretty chaotic 

Tokenized Text:  ['damn', ',', 'this', 'writing', 'was', 'pretty', 'chaotic'] 

Token IDs:  [4365, 1010, 2023, 3015, 2001, 3492, 19633]


In [ ]:
text_ids = [tokenizer.encode(text, max_length=100, padding='max_length') for text in texts]

In [ ]:
text_ids_lengths = [len(text_ids[i]) for i in range(len(text_ids))]
print(min(text_ids_lengths))
print(max(text_ids_lengths))


100
100


In [ ]:
att_masks = []
for ids in text_ids:
    masks = [int(id > 0) for id in ids]
    att_masks.append(masks)

In [ ]:
## Convert to tensor
import torch
test_x = torch.tensor(text_ids);
test_m = torch.tensor(att_masks);

## Load Model

In [ ]:
from transformers import BertForSequenceClassification, BertConfig
out_dir = './Models/fine-tuned-bert'

model = BertForSequenceClassification.from_pretrained(out_dir)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32
test_data = TensorDataset(test_x, test_m)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

outputs = []
with torch.no_grad():
    model.eval()
    for k, (mb_x, mb_m) in enumerate(test_dataloader):
        mb_x = mb_x.to(device)
        mb_m = mb_m.to(device)
        output = model(mb_x, attention_mask=mb_m)
        outputs.append(output[0].to('cpu'))

outputs = torch.cat(outputs)


In [ ]:
_, predicted_values = torch.max(outputs, 1)
predicted_values = predicted_values.numpy()


In [ ]:
predicted_values.shape

(2000,)

In [ ]:
def return_label(n):
  if n==1:
    return "sexist"
  return "not sexist"

predicted_labels = list(map(return_label, predicted_values))

In [ ]:
dfa["label_pred"] = predicted_labels

In [ ]:
dfa[["rewire_id", "label_pred"]].to_csv("Results/1b_bert.csv", index=False)